# 📓 POC Data Lakehouse Binance avec MinIO, DuckDB, Polars et Iceberg

## 1. Introduction & Objectifs

Dans ce notebook, nous construisons un **pipeline de données moderne (Lakehouse)** en utilisant :

- **MinIO** (S3-compatible) pour le stockage objet  
- **Polars** pour le traitement des fichiers CSV  
- **DuckDB** pour la transformation et l’écriture en Parquet partitionné  

### Architecture du pipeline

- **bronze** : stockage brut (`.zip` tels que téléchargés depuis Binance)  
- **silver** : données raffinées au format **Parquet**, partitionnées par `year/month/day`  
- **gold** : tables **Iceberg**, ajoutant les métadonnées pour time travel, gouvernance et gestion des snapshots  

---

---

## 2. Configuration MinIO
### Démarrer ou redémarrer MinIO

- sudo systemctl start minio   # pour lancer
- sudo systemctl restart minio # si déjà lancé mais bloqué
- sudo systemctl status minio  # vérifier que ça tourne

### Supprimer un bucket et son contenu :
- mc rb --force myminio/nom_du_bucket




Nous partons du principe que **MinIO tourne en mode standalone**.  
Exemple de lancement :

```bash
minio server /media/giujorge/Stockage/DATA/minio-data --console-address ":9001"
```

---

### commandes utiles Minio
- mc rm --recursive --force minio/bronze

- mc alias list

- mc ls minio


---


In [1]:
from minio import Minio
import os

# Config MinIO
MINIO_ENDPOINT = "localhost:9000"
MINIO_ACCESS_KEY = os.getenv("MINIO_ROOT_USER", "minioadm")
MINIO_SECRET_KEY = os.getenv("MINIO_ROOT_PASSWORD", "minioadm")

client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)


---

## 3. Création des buckets (zones du data lake)

Nous créons trois buckets suivant les bonnes pratiques :

- **bronze** : fichiers bruts  
- **silver** : Parquet partitionnés  
- **gold** : tables Iceberg  

---


In [2]:
for bucket in ["bronze", "silver", "gold"]:
    if not client.bucket_exists(bucket):
        client.make_bucket(bucket)
        print(f"Bucket {bucket} créé ✅")
    else:
        print(f"Bucket {bucket} déjà existant")


Bucket bronze déjà existant
Bucket silver déjà existant
Bucket gold déjà existant


### vérifiaction création fichiers

- ```mc alias set minio http://127.0.0.1:9000 ton_access_key ton_secret_key```
- ```mc ls minio```

---

## 4. Sauvegarde raffinée en *bronze* (Parquet partitionné)

Nous utilisons **DuckDB** pour écrire en Parquet partitionné (`year/month/day`) dans le bucket *bronze*.  

---

## 🚀 Version Optimisée - Pipeline de Traitement des Données

Cette version réécrite présente plusieurs améliorations :

### ✨ **Améliorations apportées**

1. **Architecture orientée objet** : Code structuré avec la classe `BinanceDataProcessor`
2. **Gestion d'erreurs robuste** : Meilleure gestion des exceptions et nettoyage automatique
3. **Détection automatique des timestamps** : Identification automatique des unités (s/ms/µs)
4. **Code modulaire** : Chaque étape est dans une méthode séparée pour faciliter la maintenance
5. **Configuration centralisée** : Tous les paramètres dans un dictionnaire
6. **Rapport détaillé** : Résumé complet avec statistiques et erreurs

### 🔧 **Fonctionnalités**

- **Traitement par batch** : Traite tous les fichiers ZIP automatiquement
- **Partitionnement intelligent** : Partitionnement par `year/month/day`
- **Nettoyage automatique** : Suppression des objets temporaires DuckDB
- **Logging détaillé** : Affichage du progrès et des erreurs
- **Validation des données** : Vérification des fenêtres temporelles

### 📊 **Utilisation**

Le code s'exécute automatiquement avec la configuration par défaut. Pour personnaliser, modifiez le dictionnaire `config`.

---

In [3]:
# Version corrigée avec gestion de l'écrasement
import zipfile
import datetime
import os
from pathlib import Path
import duckdb
import polars as pl

class BinanceDataProcessorV2:
    """Version corrigée avec option OVERWRITE_OR_IGNORE"""
    
    def __init__(self, config: dict):
        self.config = config
        self.csv_columns = [
            "open_time", "open", "high", "low", "close", "volume", "close_time",
            "quote_asset_volume", "number_of_trades", "taker_buy_base_volume",
            "taker_buy_quote_volume", "ignore"
        ]
        self.errors = []
        self.processed_files = 0
    
    def setup_duckdb_connection(self):
        """Configure la connexion DuckDB avec les paramètres S3"""
        con = duckdb.connect(database=":memory:")
        con.execute(f"""
            SET s3_access_key_id='{self.config["minio_access_key"]}';
            SET s3_secret_access_key='{self.config["minio_secret_key"]}';
            SET s3_endpoint='{self.config["minio_endpoint"]}';
            SET s3_url_style='path';
            SET s3_use_ssl='false';
        """)
        return con
    
    def process_file(self, zip_path: Path, con: duckdb.DuckDBPyConnection, output_path: str):
        """Traite un seul fichier de bout en bout"""
        try:
            print(f"📁 Traitement: {zip_path.name}")
            
            # 1. Lecture du ZIP
            with zipfile.ZipFile(str(zip_path)) as z:
                csv_candidates = [n for n in z.namelist() if n.lower().endswith('.csv')]
                csv_name = csv_candidates[0]
                with z.open(csv_name) as f:
                    df = pl.read_csv(f, has_header=False, new_columns=self.csv_columns)
            
            # 2. Traitement des données
            df = df.with_columns([
                pl.col("open_time").cast(pl.Int64, strict=False),
                pl.col("close_time").cast(pl.Int64, strict=False),
            ])
            
            # 3. Détection et conversion timestamp
            first_ts = df.select(pl.col("open_time").first()).item()
            if first_ts > 10_000_000_000_000:  # microsecondes
                ts_col = (pl.col("open_time") // 1000).cast(pl.Datetime("ms"))
            elif first_ts > 10_000_000_000:   # millisecondes
                ts_col = pl.col("open_time").cast(pl.Datetime("ms"))
            else:  # secondes
                ts_col = (pl.col("open_time") * 1000).cast(pl.Datetime("ms"))
            
            # 4. Ajout des colonnes finales
            ingest_id = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
            df = df.with_columns([
                ts_col.alias("datetime"),
                pl.lit(ingest_id).alias("ingest_id"),
            ]).with_columns([
                pl.col("datetime").dt.year().alias("year"),
                pl.col("datetime").dt.month().alias("month"),
                pl.col("datetime").dt.day().alias("day"),
            ])
            
            # 5. Affichage des informations
            min_max = df.select([
                pl.col("datetime").min().alias("min_dt"),
                pl.col("datetime").max().alias("max_dt"),
            ]).to_dicts()[0]
            print(f"   📊 {df.height} lignes | {min_max['min_dt']} → {min_max['max_dt']}")
            
            # 6. Sauvegarde avec gestion de l'écrasement
            try:
                con.execute("DROP VIEW IF EXISTS tmp_data")
            except:
                pass
            
            con.register("tmp_data", df.to_arrow())
            
            # COPY avec option OVERWRITE_OR_IGNORE
            sql = f"""
                COPY tmp_data
                TO '{output_path}'
                WITH (FORMAT PARQUET, PARTITION_BY (year, month, day), OVERWRITE_OR_IGNORE TRUE)
            """
            con.execute(sql)
            print(f"   ✅ Sauvegardé dans bronze")
            
            self.processed_files += 1
            
        except Exception as e:
            print(f"   ❌ Erreur: {str(e)}")
            self.errors.append({"file": zip_path.name, "error": str(e)})
        
        finally:
            try:
                con.execute("DROP VIEW IF EXISTS tmp_data")
            except:
                pass
    
    def run(self):
        """Exécute le traitement complet"""
        # Chemins
        base_path = Path(
            f"/media/giujorge/Stockage/DATA/raw/{self.config['provider']}/"
            f"{self.config['market']}/{self.config['data_frequency']}/"
            f"{self.config['data_category']}/{self.config['symbol']}/{self.config['interval']}"
        )
        
        output_path = (
            f"s3://bronze/{self.config['provider']}/{self.config['data_type']}/"
            f"{self.config['market']}/{self.config['data_frequency']}/{self.config['data_category']}/"
            f"{self.config['symbol']}/{self.config['interval']}/"
        )
        
        zip_files = sorted(base_path.glob("*.zip"))
        print(f"🚀 Début du traitement de {len(zip_files)} fichiers")
        print(f"🎯 Destination: {output_path}\n")
        
        # Connexion DuckDB
        con = self.setup_duckdb_connection()
        
        try:
            for zip_file in zip_files:
                self.process_file(zip_file, con, output_path)
        finally:
            con.close()
        
        # Résumé
        print(f"\n{'='*50}")
        print(f"✅ Fichiers traités: {self.processed_files}")
        print(f"❌ Erreurs: {len(self.errors)}")
        if self.errors:
            for error in self.errors[:5]:  # Première 5 erreurs seulement
                print(f"   • {error['file']}: {error['error']}")

# Configuration et exécution
config = {
    "provider": "binance",
    "data_type": "data", 
    "market": "spot",
    "data_frequency": "monthly",
    "data_category": "klines",
    "symbol": "BTCUSDT",
    "interval": "4h",
    "minio_access_key": os.getenv("MINIO_ROOT_USER", "minioadm"),
    "minio_secret_key": os.getenv("MINIO_ROOT_PASSWORD", "minioadm"),
    "minio_endpoint": os.getenv("MINIO_ENDPOINT", "127.0.0.1:9000"),
}

# Démarrage
processor = BinanceDataProcessorV2(config)
processor.run()

🚀 Début du traitement de 97 fichiers
🎯 Destination: s3://bronze/binance/data/spot/monthly/klines/BTCUSDT/4h/

📁 Traitement: BTCUSDT-4h-2017-08.zip
   📊 89 lignes | 2017-08-17 04:00:00 → 2017-08-31 20:00:00


/tmp/ipykernel_7197/2400263129.py:44: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  df = pl.read_csv(f, has_header=False, new_columns=self.csv_columns)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   ✅ Sauvegardé dans bronze
📁 Traitement: BTCUSDT-4h-2017-09.zip


/tmp/ipykernel_7197/2400263129.py:44: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  df = pl.read_csv(f, has_header=False, new_columns=self.csv_columns)


...

   ✅ Sauvegardé dans bronze
📁 Traitement: BTCUSDT-4h-2025-07.zip
   📊 186 lignes | 2025-07-01 00:00:00 → 2025-07-31 20:00:00


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   ✅ Sauvegardé dans bronze
📁 Traitement: BTCUSDT-4h-2025-08.zip
   📊 186 lignes | 2025-08-01 00:00:00 → 2025-08-31 20:00:00


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   ✅ Sauvegardé dans bronze

✅ Fichiers traités: 97
❌ Erreurs: 0


---

## 📋 Workflow Complet pour Nouveaux Fichiers ZIP

### **Étape 1 : Préparation**
1. **Téléchargez** vos nouveaux fichiers ZIP dans le répertoire source :
   ```
   /media/giujorge/Stockage/DATA/raw/binance/spot/monthly/klines/BTCUSDT/4h/
   ```

2. **Vérifiez** que MinIO est démarré :
   ```bash
   sudo systemctl status minio
   ```

### **Étape 2 : Traitement (Choisissez une option)**

#### 🤖 **Option A : Automatique (Recommandé)**
```python
# Traite automatiquement tous les nouveaux fichiers
incremental_processor.process_new_files_only()
```

#### 🎯 **Option B : Spécifique**
```python
# Traite uniquement les fichiers que vous spécifiez
nouveaux_fichiers = ["BTCUSDT-4h-2025-09.zip", "BTCUSDT-4h-2025-10.zip"]
incremental_processor.process_new_files_only(nouveaux_fichiers)
```

#### 🧪 **Option C : Simulation d'abord**
```python
# Voir quels fichiers seraient traités (sans les traiter)
simulate_incremental_processing()
```

### **Étape 3 : Vérification**

Après traitement, vous pouvez vérifier que les données sont bien ajoutées :

```python
# Vérifier avec DuckDB
con = duckdb.connect()
con.execute("SET s3_endpoint='127.0.0.1:9000'")  # Config MinIO
result = con.execute("""
    SELECT 
        COUNT(*) as total_records,
        MIN(datetime) as earliest_date,
        MAX(datetime) as latest_date
    FROM read_parquet('s3://bronze/binance/data/spot/monthly/klines/BTCUSDT/4h/**/*.parquet')
""").fetchone()
print(f"📊 Total: {result[0]} enregistrements | {result[1]} → {result[2]}")
```

### **🔄 Processus Récurrent**

Pour automatiser l'ajout de nouveaux fichiers, vous pouvez :

1. **Script automatisé** : Créer un script Python qui exécute `process_new_files_only()`
2. **CRON job** : Programmer l'exécution automatique (quotidienne/hebdomadaire)
3. **Workflow CI/CD** : Intégrer dans votre pipeline de données


---

✅ **Votre data lakehouse est maintenant prêt pour l'ajout incrémental de nouvelles données !**

---

### ✨ **Avantages du Processeur Incrémental**

- **🚀 Rapidité** : Ne traite que les nouveaux fichiers
- **🛡️ Sécurité** : Évite de retraiter les données existantes  
- **📊 Traçabilité** : Chaque fichier est identifié dans les métadonnées
- **🔄 Idempotent** : Peut être relancé sans risque
- **⚡ Efficace** : Utilise les partitions existantes

---

### 🗓️ **Cas d'Usage Typiques**

1. **Données mensuelles** : Ajout automatique du fichier du mois en cours
2. **Backfill historique** : Ajout de données manquantes spécifiques
3. **Mise à jour régulière** : Traitement automatique dans un CRON job
4. **Tests** : Traitement de fichiers de test sans impacter la production

---

In [1]:
"""
Guide pour ajouter de nouveaux fichiers ZIP au Data Lakehouse
"""

# 1. MÉTHODE RECOMMANDÉE : Traitement incrémental
# Utilisez cette approche pour traiter uniquement les nouveaux fichiers

import zipfile
import datetime
import os
from pathlib import Path
import duckdb
import polars as pl

class IncrementalProcessor:
    """Processeur incrémental pour nouveaux fichiers uniquement"""
    
    def __init__(self, config: dict):
        self.config = config
        self.csv_columns = [
            "open_time", "open", "high", "low", "close", "volume", "close_time",
            "quote_asset_volume", "number_of_trades", "taker_buy_base_volume",
            "taker_buy_quote_volume", "ignore"
        ]
    
    def setup_duckdb_connection(self):
        """Configure DuckDB pour S3/MinIO"""
        con = duckdb.connect(database=":memory:")
        con.execute(f"""
            SET s3_access_key_id='{self.config["minio_access_key"]}';
            SET s3_secret_access_key='{self.config["minio_secret_key"]}';
            SET s3_endpoint='{self.config["minio_endpoint"]}';
            SET s3_url_style='path';
            SET s3_use_ssl='false';
        """)
        return con
    
    def get_processed_files_list(self, con):
        """Récupère la liste des fichiers déjà traités depuis les métadonnées"""
        output_path = (
            f"s3://bronze/{self.config['provider']}/{self.config['data_type']}/"
            f"{self.config['market']}/{self.config['data_frequency']}/{self.config['data_category']}/"
            f"{self.config['symbol']}/{self.config['interval']}/**/*.parquet"
        )
        
        try:
            # Lire les métadonnées pour identifier les fichiers sources déjà traités
            # Note: Cette approche nécessite que ingest_id contienne l'info du fichier source
            result = con.execute(f"""
                SELECT DISTINCT 
                    split_part(ingest_id, '_', -1) as source_file
                FROM read_parquet('{output_path}')
                WHERE ingest_id IS NOT NULL
            """).fetchall()
            return [row[0] for row in result] if result else []
        except Exception as e:
            print(f"⚠️ Impossible de lire les métadonnées existantes: {e}")
            return []
    
    def process_new_files_only(self, specific_files=None):
        """Traite uniquement les nouveaux fichiers"""
        # Chemins
        base_path = Path(
            f"/media/giujorge/Stockage/DATA/raw/{self.config['provider']}/"
            f"{self.config['market']}/{self.config['data_frequency']}/"
            f"{self.config['data_category']}/{self.config['symbol']}/{self.config['interval']}"
        )
        
        output_path = (
            f"s3://bronze/{self.config['provider']}/{self.config['data_type']}/"
            f"{self.config['market']}/{self.config['data_frequency']}/{self.config['data_category']}/"
            f"{self.config['symbol']}/{self.config['interval']}/"
        )
        
        # Connexion DuckDB
        con = self.setup_duckdb_connection()
        
        try:
            # Option 1: Traiter des fichiers spécifiques
            if specific_files:
                files_to_process = [base_path / f for f in specific_files if (base_path / f).exists()]
                print(f"🎯 Traitement de fichiers spécifiques: {specific_files}")
            else:
                # Option 2: Traitement automatique des nouveaux fichiers
                all_files = list(base_path.glob("*.zip"))
                processed_files = self.get_processed_files_list(con)
                
                files_to_process = [
                    f for f in all_files 
                    if f.name not in processed_files
                ]
                print(f"📁 {len(all_files)} fichiers totaux, {len(processed_files)} déjà traités")
                print(f"🆕 {len(files_to_process)} nouveaux fichiers à traiter")
            
            if not files_to_process:
                print("✅ Aucun nouveau fichier à traiter!")
                return
            
            # Traitement
            processed_count = 0
            for zip_file in files_to_process:
                print(f"\n📁 Traitement: {zip_file.name}")
                success = self.process_single_file(zip_file, con, output_path)
                if success:
                    processed_count += 1
            
            print(f"\n🎉 Traitement terminé: {processed_count}/{len(files_to_process)} fichiers")
            
        finally:
            con.close()
    
    def process_single_file(self, zip_path: Path, con, output_path: str) -> bool:
        """Traite un seul fichier ZIP"""
        try:
            # 1. Lecture
            with zipfile.ZipFile(str(zip_path)) as z:
                csv_name = [n for n in z.namelist() if n.lower().endswith('.csv')][0]
                with z.open(csv_name) as f:
                    df = pl.read_csv(f, has_header=False, new_columns=self.csv_columns)
            
            # 2. Transformation
            df = df.with_columns([
                pl.col("open_time").cast(pl.Int64, strict=False),
                pl.col("close_time").cast(pl.Int64, strict=False),
            ])
            
            # 3. Timestamp conversion
            first_ts = df.select(pl.col("open_time").first()).item()
            if first_ts > 10_000_000_000_000:
                ts_col = (pl.col("open_time") // 1000).cast(pl.Datetime("ms"))
            elif first_ts > 10_000_000_000:
                ts_col = pl.col("open_time").cast(pl.Datetime("ms"))
            else:
                ts_col = (pl.col("open_time") * 1000).cast(pl.Datetime("ms"))
            
            # 4. Métadonnées avec référence au fichier source
            ingest_id = f"{datetime.datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}_{zip_path.name}"
            
            df = df.with_columns([
                ts_col.alias("datetime"),
                pl.lit(ingest_id).alias("ingest_id"),
            ]).with_columns([
                pl.col("datetime").dt.year().alias("year"),
                pl.col("datetime").dt.month().alias("month"),
                pl.col("datetime").dt.day().alias("day"),
            ])
            
            # 5. Sauvegarde
            try:
                con.execute("DROP VIEW IF EXISTS tmp_new_data")
            except:
                pass
            
            con.register("tmp_new_data", df.to_arrow())
            
            sql = f"""
                COPY tmp_new_data
                TO '{output_path}'
                WITH (FORMAT PARQUET, PARTITION_BY (year, month, day), OVERWRITE_OR_IGNORE TRUE)
            """
            con.execute(sql)
            
            min_max = df.select([
                pl.col("datetime").min().alias("min_dt"),
                pl.col("datetime").max().alias("max_dt"),
            ]).to_dicts()[0]
            
            print(f"   ✅ {df.height} lignes | {min_max['min_dt']} → {min_max['max_dt']}")
            return True
            
        except Exception as e:
            print(f"   ❌ Erreur: {str(e)}")
            return False
        finally:
            try:
                con.execute("DROP VIEW IF EXISTS tmp_new_data")
            except:
                pass

# Configuration (même que précédemment)
config = {
    "provider": "binance",
    "data_type": "data",
    "market": "spot", 
    "data_frequency": "monthly",
    "data_category": "klines",
    "symbol": "BTCUSDT",
    "interval": "4h",
    "minio_access_key": os.getenv("MINIO_ROOT_USER", "minioadm"),
    "minio_secret_key": os.getenv("MINIO_ROOT_PASSWORD", "minioadm"),
    "minio_endpoint": os.getenv("MINIO_ENDPOINT", "127.0.0.1:9000"),
}

# Instanciation du processeur incrémental
incremental_processor = IncrementalProcessor(config)

print("🔄 PROCESSEUR INCRÉMENTAL PRÊT")
print("Utilisez les méthodes suivantes selon vos besoins:")
print("1. incremental_processor.process_new_files_only()  # Auto-détection")
print("2. incremental_processor.process_new_files_only(['nouveau_fichier.zip'])  # Fichiers spécifiques")

🔄 PROCESSEUR INCRÉMENTAL PRÊT
Utilisez les méthodes suivantes selon vos besoins:
1. incremental_processor.process_new_files_only()  # Auto-détection
2. incremental_processor.process_new_files_only(['nouveau_fichier.zip'])  # Fichiers spécifiques


---

## 🔄 Processus pour Ajouter de Nouveaux Fichiers ZIP

Maintenant que votre data lakehouse est configuré, voici **3 approches** pour ajouter de nouveaux fichiers :

### 🎯 **Option 1 : Traitement Automatique (Recommandé)**
```python
# Détecte automatiquement les nouveaux fichiers et les traite
incremental_processor.process_new_files_only()
```

### 📂 **Option 2 : Fichiers Spécifiques**
```python
# Traite uniquement les fichiers que vous spécifiez
nouveaux_fichiers = ["BTCUSDT-4h-2025-09.zip", "BTCUSDT-4h-2025-10.zip"]
incremental_processor.process_new_files_only(nouveaux_fichiers)
```

### 🔧 **Option 3 : Workflow Manuel**
1. **Téléchargez** vos nouveaux fichiers ZIP dans le répertoire source
2. **Exécutez** une des commandes ci-dessus
3. **Vérifiez** que les données sont bien ajoutées dans MinIO

In [13]:
# 🧪 EXEMPLE DE TEST - Traitement incrémental
# Décommentez et exécutez pour tester

# Exemple 1: Traitement automatique de tous les nouveaux fichiers
# incremental_processor.process_new_files_only()

# Exemple 2: Traitement de fichiers spécifiques (remplacez par vos vrais fichiers)
# nouveaux_fichiers = ["BTCUSDT-4h-2025-09.zip"]  # Vos nouveaux fichiers
# incremental_processor.process_new_files_only(nouveaux_fichiers)

# Exemple 3: Simulation - vérifier quels fichiers seraient traités
def simulate_incremental_processing():
    """Simule le traitement pour voir quels fichiers seraient traités"""
    base_path = Path(
        f"/media/giujorge/Stockage/DATA/raw/{config['provider']}/"
        f"{config['market']}/{config['data_frequency']}/"
        f"{config['data_category']}/{config['symbol']}/{config['interval']}"
    )
    
    all_files = list(base_path.glob("*.zip"))
    print(f"📁 Fichiers trouvés dans le répertoire source:")
    for i, f in enumerate(sorted(all_files)[:10], 1):  # Afficher les 10 premiers
        print(f"   {i:2d}. {f.name}")
    
    if len(all_files) > 10:
        print(f"   ... et {len(all_files) - 10} autres fichiers")
    
    print(f"\n🔍 Pour identifier les nouveaux fichiers, le processeur va:")
    print(f"   1. Lire les métadonnées depuis MinIO")
    print(f"   2. Comparer avec les fichiers locaux")
    print(f"   3. Traiter uniquement les différences")
    
    return len(all_files)

# Lancez cette simulation pour voir l'état actuel
total_files = simulate_incremental_processing()
print(f"\n💡 Prêt à traiter jusqu'à {total_files} fichiers si nécessaire!")

📁 Fichiers trouvés dans le répertoire source:
    1. BTCUSDT-4h-2017-08.zip
    2. BTCUSDT-4h-2017-09.zip
    3. BTCUSDT-4h-2017-10.zip
    4. BTCUSDT-4h-2017-11.zip
    5. BTCUSDT-4h-2017-12.zip
    6. BTCUSDT-4h-2018-01.zip
    7. BTCUSDT-4h-2018-02.zip
    8. BTCUSDT-4h-2018-03.zip
    9. BTCUSDT-4h-2018-04.zip
   10. BTCUSDT-4h-2018-05.zip
   ... et 87 autres fichiers

🔍 Pour identifier les nouveaux fichiers, le processeur va:
   1. Lire les métadonnées depuis MinIO
   2. Comparer avec les fichiers locaux
   3. Traiter uniquement les différences

💡 Prêt à traiter jusqu'à 97 fichiers si nécessaire!
